# 基本的な文法

## 基本的な値

Elixirは動的型付け言語ですが, 内部的には型が存在します。

[基本的な型](https://elixir-lang.org/getting-started/basic-types.html)は以下。

- Integer
- Float
- Boolean
- String
- List
- Tuple
- Atom
- Anonymous function
- Port
- Reference
- PID

In [ ]:
# integer
IO.inspect 1
IO.inspect 0x01 # 16進数

In [ ]:
# float
IO.inspect 1.0
IO.inspect 1.0e+3

In [ ]:
# boolean
IO.inspect true
IO.inspect false

In [ ]:
# string
IO.inspect "abcABC"
IO.inspect "マルチバイト文字列"

### List (linked list)

- 値が0個以上あることを表現する
- 要素の型はどんな組み合わせでもいい
- 要素数が動的に増減する場合に使用する
- 内部実装はlinked list

In [ ]:
# list

IO.inspect [0, "a", true, [0, 1, 2]]

l = [0, 1, 2]
IO.inspect l

try do
  IO.inspect l[0]
rescue
  ArgumentError -> IO.inspect "Access to an item by index failed"
end

Listに関係して、Elixirには文字を取り扱う方法が2つあり、それらは明確に異なる。
- ダブルクォーテーション`"`を使う場合 => 文字列
- シングルクォーテーション`'`を使う場合 => 文字のリスト

基本的にはダブルクォーテーションを使うのでいいが、

In [ ]:
# コードポイントのリスト
IO.puts 'ABC'
IO.puts [65, 66, 67]
IO.puts 'ABC' == [65, 66, 67]

# 文字列とは異なる型
IO.puts "ABC" != 'ABC'

### Tuple

- 複数の値を組み合わせた値
- 中身の値の型はどんな組合せでも良い
- 要素数はコンパイル時に確定する。動的に要素の追加や削除はできない
- プログラマが何らかの「意味のある値の組合せ」を表現するために使われることが多い
  - floatの組で「座標」を表現する
  - `:ok`, `:error`との組で処理の「成功」または「失敗」を表す

In [ ]:
# tuple
IO.inspect {0, 1, 2}

IO.inspect File.read("./resources/2/hello.txt")
IO.inspect File.read("./resource/not/found")

### Atom

- `:`から始まる名前付きの値で, 実体がErlangVM上で**ただ1つ**
- コンパイル時(コーディング時)に定まっている定数を表現するために使われることが多い
- 実体がただ1つなので, 同じAtomをプログラム内の複数箇所で使っても値はコピーされず, 既存のAtomに対する参照のみが増える
- 一度生成されたAtomはVMから**消えない**(Garbage Collect されない)
  - メモリを使い尽くすことにつながる恐れがあるので, 動的にAtomを生成すべきではない
  - コーディング時にAtomの使用をためらう必要はない

In [ ]:
# atom
IO.puts :atom

IO.puts :atom == :atom
IO.puts :atom != :another

# booleanのtrueやfalse, 値がないことを示すnilも実際にはatomとして定義されています
IO.puts true == :true
IO.puts nil == :nil

### Anonymous functions

- Elixirの関数は2種類
  - モジュール(module)に属する必要がある名前付き関数(named function)
  - コードのどこにでも書ける無名関数(anonymous function)
  
- 無名関数の定義方法
  - 基本
    - `fn (`args`) -> `return value` end`
  - 引数の括弧は省略可能
    - `fn `args` -> `return value` end`
- 無名関数は, 内部的には定義時に返る`#Function<43.97283095/2 in :erl_eval.expr/5>`のようなハッシュで識別されている

In [ ]:
fn (a, b) -> a + b end

- 無名関数を変数に束縛できる
- 無名関数の呼び出し時は, `add.(1, 2)`のように`.()`とカッコの前にピリオドが必要
  - 名前付き関数呼び出しの`()`が省略できる仕様との間で曖昧さを回避するため
  - `IO.puts "hello"`は`IO` moduleの名前付き関数`puts`を呼び出しているので括弧を省略できている
  - 丁寧に書くと`IO.puts("hello")`

In [ ]:
add = fn (a, b) -> a + b end
mul = fn a, b -> a * b end

IO.puts add.(1, 2)
IO.puts mul.(3, 2)

- 引数名を省略する記法もある
  - `&()`で囲み, `&1`, `&2`, ... で第1引数から順に参照する
  - 後述の高階関数に単純な関数を与える際などに使うことがある

In [ ]:
add_short = &(&1 + &2)
IO.puts add_short.(1, 2)

- 式を複数持つ無名関数を定義することもできる
  - 関数内の最後の式の値のみが返り値になる

In [ ]:
verbose_add = fn (a, b) ->
  IO.puts a
  IO.puts b
  a + b
end

verbose_add.(2, 3)

#### おまけクイズ

無名関数を変数に束縛せずに呼び出すことも可能。どうすればいいだろうか?
他言語では即時関数と呼ばれている場合がある。

In [ ]:
# Q. 変数に束縛せず2つの値を引数にとってその和を返す関数を定義し, かつその場で無名関数を1と2に適用せよ。(返り値の期待値は3)


### Map

- keyとvalueの組
- 他言語では辞書やオブジェクトと呼ばれる

In [ ]:
m = %{"key" => "value", 0 => 1, :atom => :value}

IO.puts m["key"]
IO.puts m[0]
IO.puts m[:atom]

- keyがatomのとき、省略記法が使える

In [ ]:
m = %{:a => 0, "b" => 1}
IO.puts m.a
IO.puts m[:a]
IO.puts m["b"]
# IO.puts m.b # => KeyError

IO.puts %{a: 0, b: "b", c: :value} == %{:a => 0, :b => "b", :c => :value}
IO.puts %{a: 0} == %{"a" => 0}

## 演算子; Operators

[Basic operators - Elixir](https://elixir-lang.org/getting-started/basic-operators.html)

- 中置演算子
  - すでに出ている`+`や`*`などの算術演算子
  - リストの連結演算子 `++`
  - 文字列の連結演算子 `<>`
- 中置演算子も実体は関数
  - ドキュメントでは`+/2`のように参照する

In [ ]:
1 + 2

In [ ]:
[ 1, 2, 3] ++ [4, 5, 6]

In [ ]:
"Hello" <> " world!"